In [ ]:
#WOA and GWO
#linear hybrid(first woa then gwo)

In [1]:
#import all the necessary libraries
import networkx as nx
import numpy as np
import pandas as pd
import networkx.algorithms.community as nx_comm
import math
import matplotlib.pyplot as plt
import random
print("Libraries successfully imported")

#all the side functions
def generate_chrom(nodes_length,Adj):
    chrom = np.array([],dtype=int)
    for x in range(nodes_length):
        rand = np.random.randint(0,nodes_length)
        while Adj[x,rand] != 1:
            rand = np.random.randint(0,nodes_length)
        chrom = np.append(chrom,rand)
    return chrom

def merge_subsets(sub):
    arr =[]
    to_skip=[]
    for s in range(len(sub)):
        if sub[s] not in to_skip:
            new = sub[s]
            for x in sub:
                if sub[s] & x:
                    new = new | x
                    to_skip.append(x)
            arr.append(new)
    return arr

def find_subsets(chrom):
    sub = [{x,chrom[x]} for x in range(len(chrom))]
    result=sub
    i=0
    while i<len(result):
        candidate = merge_subsets(result)
        if candidate != result:
            result = candidate
        else:
            break
        result=candidate
        i+=1
    return result

def community_score(chrom,subsets,r,Adj):
    matrix = Adj.toarray()
    CS=0
    for s in subsets:
        submatrix = np.zeros((len(chrom),len(chrom)),dtype=int)
        for i in s:
            for j in s:
                submatrix[i][j]=matrix[i][j]
        M=0
        v=0
        PS=0
        for row in list(s):
            ki = np.sum(matrix[row])
            kiin = np.sum(submatrix[row])
            kiout = ki - kiin
            P= kiin/ki
            PS+=P
            row_mean = kiin/len(s)
            v+=np.sum(submatrix[row])
            M+=(row_mean**r)/len(s)
        CS+=M*v
    OS= 0.5*CS/len(subsets) + 0.5*(1/PS)*len(subsets)  #Overall score is calculated by maximizing CS and min PS
    return OS

def roulette_selection(df_elites):
    prob = np.random.random_sample()
    sum_cs=np.sum(df_elites["community_score"])
    x=0
    selected = 0
    for i in df_elites.index:
        x += df_elites["community_score"][i]
        X=x/sum_cs
        if prob < X:
            chosen=i
            break
    return chosen

def uniform_crossover_GE(parent_1,parent_2,crossover_rate):
    if np.random.random_sample() < crossover_rate:
        length = len(parent_1)
        mask = np.random.randint(2, size=length)
        child = np.zeros(length,dtype=int)
        for i in range(len(mask)):
            if mask[i] == 1:
                child[i]=parent_1[i]
            else:
                child[i]=parent_2[i]
        return child
    else:
        return np.array([])

def crossover_DE(parent,trial,crossover_rate):
    if np.random.random_sample() >= crossover_rate:
        length = len(parent['chrom'])
        mask = np.random.randint(2, size=length)
        child = np.zeros(length,dtype=int)
        for i in range(len(mask)):
            if mask[i] == 1:
                child[i]=trial[i]
            else:
                child[i]=parent['chrom'][i]
        return child
    else:
        return trial
    
def mutation_GE(chrom,Adj,mutation_rate):
    if np.random.random_sample() < mutation_rate:
        chrom = chrom
        neighbor = []
        while len(neighbor) < 2:
            mutant = np.random.randint(1,len(chrom))
            row = Adj[mutant].toarray()[0]
            neighbor = [i for i in range(len(row)) if row[i]==1]
            if len(neighbor) > 1:
                neighbor.remove(chrom[mutant])
                to_change=int(np.floor(np.random.random_sample()*(len(neighbor))))
                chrom[mutant]=neighbor[to_change]
                neighbor.append(chrom[mutant])
    return chrom

def mutation_DE(parent , target , rand1 , rand2 ,Adj,mutation_rate):
    trial = []
    for i in range(0,len(parent['chrom'])):
        temp = target['chrom'][i] + mutation_rate*(rand1['chrom'][i] - rand2['chrom'][i])
        temp = int(temp)
        if temp < 0 or temp >= Adj.shape[0] or Adj[i,temp]==0:
            temp = target['chrom'][i]
        trial.append(temp)
    return trial

#NMI
def entropy(nums):
    z = np.bincount(nums)
    N = len(nums)
    assert nums.shape == (N, )
    ent = 0.0
    for e in z:
        if e != 0:
            p = float(e) / N
            ent += p*math.log(p)
    assert ent <= 0
    ent = -ent
    assert ent >=0
    return ent

def computeNMI(clusters, classes):
    assert clusters.shape == classes.shape
    A = np.c_[(clusters, classes)]
    A = np.array(A)
    N = A.shape[0]
    assert A.shape == (N, 2)
    H_clusters = entropy(A[:, 0])
    H_classes = entropy(A[:, 1])
    NMI = 0.0
    for k in np.unique(A[:, 0]):
        z = A[A[:, 0] == k, 1]
        len_wk = len(z)
        t = A[:, 1]
        for e in np.unique(z):
            wk_cj=len(z[z==e])
            len_cj=len(t[t == e])
            assert wk_cj <= len_cj
            numerator= (float(wk_cj) / float(N)) * math.log( (N*wk_cj) / float(len_wk * len_cj)  )
            NMI += numerator
    NMI /= float((H_clusters + H_classes) * 0.5)

    assert (NMI > 0.0 or abs(NMI) < 1e-10) and (NMI < 1.0 or abs(NMI - 1.0) < 1e-10)
    return NMI

def check_res(res,Adj):
    print(res)
    for i in range(0,len(res)):
        if Adj[i,res[i]] ==0:
            return False
    return True

def print_graph(graph, result, nodes):
    colors = []
    for i in range(0, len(result)):
        colors.append('#%06X' % np.random.randint(0, 0xFFFFFF))
    col_arr = ['']*len(nodes)
    for i in range(0,len(result)):
        for j in result[i]:
            col_arr[j-1] = colors[i]
    nx.draw(graph, with_labels=True,node_color = col_arr)
    plt.show()
    
def ShrinkingEncircling(i,P_se,Adj):
    
    N = Adj.shape[0]
    n = N*P_se
    color = np.zeros(N,dtype=int)
    val = 0
    for j in i["subsets"]:
        for k in j:
            color[k] = val
        val+=1
    
    
        
    for z in range(0,int(n)):
        index = random.randrange(0,N)
        freq = np.zeros(val, dtype = int)
        for k in range(0, N):
            if Adj[index,k] == 1:
                freq[color[k]]+=1
        max_freq = np.argmax(freq)
        
        for k in range(0, len(color)):
            if(color[k] == max_freq and Adj[index,k] == 1):
                i["chrom"][index] = k
                break
    i["subsets"] = find_subsets(i["chrom"])
    i["community_score"] = community_score(i["chrom"],i["subsets"],1.5,Adj)
    return i

def RandomSearchingOperation(i,P_rs,Adj):
    neigh = []
#     color = AssignColor(i,Adj)
    for j in i["chrom"]:
        r1 = np.random.rand()
        if r1<=P_rs:
            for k in range(0,Adj.shape[0]-1):
                if Adj[j,k]==1:
                    if j!=k:
                        neigh.append(k)
            if len(neigh)!=0:
                ind = random.randrange(0,len(neigh))
                i["chrom"][j] = neigh[ind]
                neigh.clear()
    i["subsets"] = find_subsets(i["chrom"])
    i["community_score"] = community_score(i["chrom"],i["subsets"],1.5,Adj)
    return i   
           

    
def SpiralUpdatingOperation(i,P_su,gbest,Adj):
    N = len(gbest["chrom"])
    n = N*P_su
    for k in range(0,int(n)):
        r = random.randrange(0,N)
        i["chrom"][r] = gbest["chrom"][r]
    
    i["subsets"] = find_subsets(i["chrom"])
    i["community_score"] = community_score(i["chrom"],i["subsets"],1.5,Adj)
    return i
    


print("All functions are ready to use")

Libraries successfully imported
All functions are ready to use


In [10]:
def WOA(nodes,edges,population=20,generation=100,P_se=0.05, P_su = 0.05, P_rs = 0.1):
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],1.5,Adj),axis=1)
    population_count = population
    gen = 0
    globalbest=[]
    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
#     ShrinkingEncircling(dframe.loc[0],P_se, Adj)
    while gen < generation:
       
        for i in range(int(np.floor(population/10))):
            elites = dframe.sort_values("community_score",ascending=True)[int(np.floor(population/10)):]
            best3 = dframe.sort_values("community_score", ascending=False)[0:3]
            xalpha = dframe.loc[best3.index[0]]['chrom']
            xbeta = dframe.loc[best3.index[1]]['chrom']
            xgamma = dframe.loc[best3.index[2]]['chrom']
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            if r < 0.5:
                if abs(M) < 1:
                    dframe.loc[i] = ShrinkingEncircling(dframe.loc[i],P_se, Adj)
                elif abs(M)>=1:
                    dframe.loc[i] = RandomSearchingOperation(dframe.loc[i],P_rs,Adj)
#                 dframe.loc[i] = gwolagao(xalpha,xbeta,xgamma,dframe.loc[i], Adj)
            elif r>=0.5:
                dframe.loc[i] = SpiralUpdatingOperation(dframe.loc[i],P_su,globalbest,Adj)
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   
   


    res = globalbest
    fs = res['community_score']
    nodes_subsets = res["subsets"]
    res = res['chrom']
#     istrue = check_res(res,Adj)
#     print(istrue)
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = globalbest
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity, fs
print("WOA loaded to use")


def gwolagao(xalpha, xbeta, xgamma, wolf, Adj,a):
    A1 = (2*a*np.random.rand()) - a
    C1 = 2*np.random.rand()
    Dalpha = abs(C1*xalpha - wolf['chrom'])
    X1 = abs(xalpha - (A1*Dalpha))
    A2 = (2*a*np.random.rand()) - a
    C2 = 2*np.random.rand()
    Dbeta = abs(C2*xbeta - wolf['chrom'])
    X2 = abs(xbeta - (A2*Dbeta))
    A3 = (2*a*np.random.rand()) - a
    C3 = 2*np.random.rand()
    Dgamma = abs(C3*xgamma - wolf['chrom'])
    X3 = abs(xgamma - (A3*Dgamma))
    Xnew = (X1+X2+X3)/3
    Xnew = np.array([int(i) for i in Xnew])
    for i in range(0,len(Xnew)):
        if Xnew[i] < 0 or Xnew[i] >= Adj.shape[0] or Adj[i,Xnew[i]]==0:
            Xnew[i] = xalpha[i]
    wolf["chrom"] = Xnew
    wolf["subsets"]= find_subsets(Xnew)
    wolf["community_score"] = community_score(wolf["chrom"],wolf["subsets"], 1.5, Adj)
    return wolf



WOA loaded to use


In [3]:
def GWOWOAlinear(nodes,edges,population=20,generation=100,r=1.5,P_se=0.05, P_su = 0.05, P_rs = 0.1):
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],r,Adj),axis=1)
    gen = 0
    globalbest=[]
    globalbest = dframe.sort_values("community_score",ascending=False).index[0]
    globalbest = dframe.loc[globalbest]
    population_count = population
    while gen < generation:
       
        for i in range(int(np.floor(population/10))):
            elites = dframe.sort_values("community_score",ascending=True)[int(np.floor(population/10)):]
            best3 = dframe.sort_values("community_score", ascending=False)[0:3]
            xalpha = dframe.loc[best3.index[0]]['chrom']
            xbeta = dframe.loc[best3.index[1]]['chrom']
            xgamma = dframe.loc[best3.index[2]]['chrom']
            
            r = np.random.rand()
            a = (2-2*gen)/generation
            M = (2*r-1)*a 
            if r < 0.5:
                if abs(M) < 1:
                    dframe.loc[i] = ShrinkingEncircling(dframe.loc[i],P_se, Adj)
                elif abs(M)>=1:
                    dframe.loc[i] = RandomSearchingOperation(dframe.loc[i],P_rs,Adj)

            elif r>=0.5:
#                 dframe.loc[i] = SpiralUpdatingOperation(dframe.loc[i],P_su,globalbest,Adj)
                dframe.loc[i] = gwolagao(xalpha,xbeta,xgamma,dframe.loc[i], Adj, a)
        tempbest = dframe.sort_values("community_score",ascending=False).index[0]
        tempbest = dframe.loc[tempbest]
        
        if globalbest["community_score"]<tempbest["community_score"]:
            globalbest = tempbest
                           
        gen +=1   
    sorted_df = dframe.sort_values("community_score",ascending=False).index[0]
    res = dframe.loc[sorted_df]
    res = res['chrom']
    res_subsets = find_subsets(res)
    nodes_subsets = res_subsets
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = dframe.loc[sorted_df]
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI, modularity
print("GWOWOAlinear ready to use")

GWOWOAlinear ready to use


In [8]:
def GWO(nodes,edges,population=20,generation=100,r=1.5):
    graph=nx.Graph() 
    graph.add_nodes_from(nodes) #adds nodes
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph) 
    nodes_length = len(graph.nodes())
#     nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],r,Adj),axis=1)
    gen = 0
    population_count = population
    while gen < generation:
        for i in range(int(np.floor(population/10))):
            elites = dframe.sort_values("community_score",ascending=True)[int(np.floor(population/10)):]
            best3 = dframe.sort_values("community_score", ascending=False)[0:3]
            xalpha = dframe.loc[best3.index[0]]['chrom']
            xbeta = dframe.loc[best3.index[1]]['chrom']
            xgamma = dframe.loc[best3.index[2]]['chrom']
            a = 2*(1-(gen/generation))
            for index , wolf in elites.iterrows():
                A1 = (2*a*np.random.rand()) - a
                C1 = 2*np.random.rand()
                Dalpha = abs(C1*xalpha - wolf['chrom'])
                X1 = abs(xalpha - (A1*Dalpha))
                A2 = (2*a*np.random.rand()) - a
                C2 = 2*np.random.rand()
                Dbeta = abs(C2*xbeta - wolf['chrom'])
                X2 = abs(xbeta - (A2*Dbeta))
                A3 = (2*a*np.random.rand()) - a
                C3 = 2*np.random.rand()
                Dgamma = abs(C3*xgamma - wolf['chrom'])
                X3 = abs(xgamma - (A3*Dgamma))
                Xnew = (X1+X2+X3)/3
                Xnew = np.array([int(i) for i in Xnew])
                for i in range(0,len(Xnew)):
                    if Xnew[i] < 0 or Xnew[i] >= Adj.shape[0] or Adj[i,Xnew[i]]==0:
                        Xnew[i] = xalpha[i]
                subsetnew = find_subsets(Xnew)
                fscorenew = community_score(Xnew,subsetnew, r, Adj)
                dframe.loc[population_count]=[Xnew,subsetnew,fscorenew]
                population_count += 1
        dfsorted = dframe.sort_values("community_score",ascending=False)
        to_drop = dfsorted.index[population:]
        dframe.drop(to_drop,inplace=True)
        gen +=1   
    sorted_df = dframe.sort_values("community_score",ascending=False).index[0]
    res = dframe.loc[sorted_df]
    fs = res["community_score"]
    res = res['chrom']
#     istrue = check_res(res,Adj)
#     print(istrue)
    res_subsets = find_subsets(res)
    nodes_subsets = res_subsets
    nodes_list = list(graph.nodes())
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    inbuilt_NMI = 0
    
    clu = dframe.loc[sorted_df]
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
#     for index, target in dframe.iterrows():
#         temp = np.array(target['chrom'])
#         x = met.normalized_mutual_info_score(clu,temp)
#         inbuilt_NMI += x
    
#     inbuilt_NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
    return result, NMI , modularity, fs
print("GWO loaded to use")

GWO loaded to use


In [4]:
G = nx.read_gml('dolphins.gml', label='id',destringizer=int)
nodes = np.array(G.nodes)
edges = np.array(G.edges)
print("Data ready")

Data ready


In [5]:
NMI_arr_GWOWOAlinear = []
mod_arr_GWOWOAlinear = []
bst_arr_GWOWOAlinear = []

for i in range(0,10):
    print("GWOWOAlinear",i+1)
    best_community, NMI, modularity = GWOWOAlinear(nodes,edges)
    NMI_arr_GWOWOAlinear.append(NMI)
    mod_arr_GWOWOAlinear.append(modularity)
    bst_arr_GWOWOAlinear.append(best_community)
    
NMI_avg_GWOWOAlinear = sum(NMI_arr_GWOWOAlinear)/len(NMI_arr_GWOWOAlinear)
mod_avg_GWOWOAlinear = sum(mod_arr_GWOWOAlinear)/len(mod_arr_GWOWOAlinear)
NMI_bst_GWOWOAlinear = max(NMI_arr_GWOWOAlinear)  #best
mod_bst_GWOWOAlinear = max(mod_arr_GWOWOAlinear)
NMI_lst_GWOWOAlinear = min(NMI_arr_GWOWOAlinear)  #least
mod_lst_GWOWOAlinear = min(mod_arr_GWOWOAlinear)


print("GWOWOAlinear")
print('NMI avg: ', NMI_avg_GWOWOAlinear)
print('NMI max: ', NMI_bst_GWOWOAlinear)
print('NMI min: ', NMI_lst_GWOWOAlinear)
print()
print('mod avg: ', mod_avg_GWOWOAlinear)
print('mod max: ', mod_bst_GWOWOAlinear)
print('mod min: ', mod_lst_GWOWOAlinear)
print("Done")

GWOWOAlinear 1


C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\288747535.py:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj = nx.adjacency_matrix(graph)
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\2299466250.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["subsets"] = find_subsets(i["chrom"])
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\2299466250.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["community_score"] = community_score(i["chrom"],i["subsets"],1.5,Adj)
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\503701631.py:94: SettingWithCop

GWOWOAlinear 2


C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\288747535.py:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj = nx.adjacency_matrix(graph)
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\2299466250.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["subsets"] = find_subsets(i["chrom"])
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\2299466250.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["community_score"] = community_score(i["chrom"],i["subsets"],1.5,Adj)
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\503701631.py:94: SettingWithCop

GWOWOAlinear 3


...

GWOWOAlinear 9


C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\288747535.py:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj = nx.adjacency_matrix(graph)
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\503701631.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wolf["chrom"] = Xnew
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\503701631.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wolf["subsets"]= find_subsets(Xnew)
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\503701631.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

GWOWOAlinear 10


C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\288747535.py:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj = nx.adjacency_matrix(graph)
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\2299466250.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["subsets"] = find_subsets(i["chrom"])
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\2299466250.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["community_score"] = community_score(i["chrom"],i["subsets"],1.5,Adj)
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\503701631.py:94: SettingWithCop

GWOWOAlinear
NMI avg:  0.8685538632868539
NMI max:  0.8781627442241957
NMI min:  0.8590366341726051

mod avg:  0.3336853763696056
mod max:  0.42249119892409315
mod min:  0.26976780981764964
Done


In [6]:
NMI_arr_GWO = []
mod_arr_GWO = []
bst_arr_GWO = []

for i in range(0,10):
    print("GWO",i+1)
    best_community, NMI, modularity = GWO(nodes,edges)
    NMI_arr_GWO.append(NMI)
    mod_arr_GWO.append(modularity)
    bst_arr_GWO.append(best_community)
    
NMI_avg_GWO = sum(NMI_arr_GWO)/len(NMI_arr_GWO)
mod_avg_GWO = sum(mod_arr_GWO)/len(mod_arr_GWO)
NMI_bst_GWO = max(NMI_arr_GWO)  #best
mod_bst_GWO = max(mod_arr_GWO)
NMI_lst_GWO = min(NMI_arr_GWO)  #least
mod_lst_GWO = min(mod_arr_GWO)


print("GWO")
print('NMI avg: ', NMI_avg_GWO)
print('NMI max: ', NMI_bst_GWO)
print('NMI min: ', NMI_lst_GWO)
print()
print('mod avg: ', mod_avg_GWO)
print('mod max: ', mod_bst_GWO)
print('mod min: ', mod_lst_GWO)
print("Done")

GWO 1


C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\503701631.py:103: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj = nx.adjacency_matrix(graph)


ValueError: too many values to unpack (expected 3)

In [9]:
NMI_arr_WOA = []
mod_arr_WOA = []
bst_arr_WOA = []

for i in range(0,10):
    print("WOA",i+1)
    best_community, NMI, modularity = WOA(nodes,edges)
    NMI_arr_WOA.append(NMI)
    mod_arr_WOA.append(modularity)
    bst_arr_WOA.append(best_community)
    
NMI_avg_WOA = sum(NMI_arr_WOA)/len(NMI_arr_WOA)
mod_avg_WOA = sum(mod_arr_WOA)/len(mod_arr_WOA)
NMI_bst_WOA = max(NMI_arr_WOA)  #best
mod_bst_WOA = max(mod_arr_WOA)
NMI_lst_WOA = min(NMI_arr_WOA)  #least
mod_lst_WOA = min(mod_arr_WOA)


print("WOA")
print('NMI avg: ', NMI_avg_WOA)
print('NMI max: ', NMI_bst_WOA)
print('NMI min: ', NMI_lst_WOA)
print()
print('mod avg: ', mod_avg_WOA)
print('mod max: ', mod_bst_WOA)
print('mod min: ', mod_lst_WOA)
print("Done")

WOA 1


C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\1958936274.py:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj = nx.adjacency_matrix(graph)
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\2299466250.py:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["subsets"] = find_subsets(i["chrom"])
C:\Users\SHUBH\AppData\Local\Temp\ipykernel_26100\2299466250.py:252: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["community_score"] = community_score(i["chrom"],i["subsets"],1.5,Adj)


TypeError: gwolagao() missing 1 required positional argument: 'a'